In [ ]:
!nvidia-smi

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


# Task 1. Реализовать параллельную сортировку слиянием на CUDA

In [2]:
%%writefile task_1_merge_sort.cu

#include <iostream>      // Стандартный ввод / вывод
#include <vector>        // Контейнер vector
#include <algorithm>     // min, max
#include <cuda_runtime.h>// CUDA Runtime API
#include <random>        // Генерация случайных чисел
#include <chrono>        // Измерение времени выполнения
#include <omp.h>         // OpenMP для параллелизма на CPU

using namespace std;

// ======================================================
// CUDA Kernel
// Назначение:
//   Каждый CUDA-блок сортирует свой подмассив фиксированного
//   размера, используя shared memory.
// Подход:
//   1) Копирование данных из global memory в shared memory
//   2) Сортировка внутри блока (Insertion Sort)
//   3) Запись отсортированных данных обратно в global memory
// ======================================================
__global__ void sortBlocksSharedKernel(int* data, int blockSize, int n) {

    // Выделяем shared memory динамически
    // Размер задаётся при запуске kernel
    extern __shared__ int s[];

    // Локальный индекс потока в блоке
    int tid = threadIdx.x;

    // Глобальный индекс элемента в массиве
    int gid = blockIdx.x * blockSize + tid;

    // Защита от выхода за границы массива
    if (gid < n)
        s[tid] = data[gid];
    else
        // Если элементов меньше, чем размер блока,
        // заполняем "лишние" элементы максимально возможным значением
        s[tid] = INT_MAX;

    // Синхронизация: все потоки должны загрузить данные
    __syncthreads();

    // --------------------------------------------------
    // Сортировка вставками (Insertion Sort) внутри блока
    // Подходит для небольших массивов (shared memory)
    // --------------------------------------------------
    for (int i = 1; i < blockSize; i++) {
        int key = s[i];
        int j = i - 1;

        // Сдвигаем элементы вправо, пока не найдём позицию
        while (j >= 0 && s[j] > key) {
            s[j + 1] = s[j];
            j--;
        }
        s[j + 1] = key;

        // Синхронизация после каждого шага
        __syncthreads();
    }

    // Записываем отсортированные данные обратно в global memory
    if (gid < n)
        data[gid] = s[tid];
}

// ======================================================
// CPU-функция слияния двух отсортированных подмассивов
// [left, mid) и [mid, right)
// Используется в Merge Sort
// ======================================================
void merge(int* arr, int* temp, int left, int mid, int right) {
    int i = left;   // указатель левой части
    int j = mid;    // указатель правой части
    int k = left;   // указатель во временный массив

    // Основное слияние
    while (i < mid && j < right)
        temp[k++] = (arr[i] <= arr[j]) ? arr[i++] : arr[j++];

    // Копирование оставшихся элементов
    while (i < mid)   temp[k++] = arr[i++];
    while (j < right) temp[k++] = arr[j++];
}

// ======================================================
// Параллельная версия Merge Sort с использованием OpenMP
// Алгоритм:
//   Bottom-up Merge Sort
//   На каждом шаге увеличиваем ширину сливаемых блоков
// ======================================================
void parallelMergeSortOMP(int* arr, int n) {

    // Временный массив для слияния
    int* temp = new int[n];

    // Ширина подмассивов: 1, 2, 4, 8, ...
    for (int width = 1; width < n; width *= 2) {

        // Параллельное слияние подмассивов
        #pragma omp parallel for schedule(static)
        for (int i = 0; i < n; i += 2 * width) {
            int left  = i;
            int mid   = min(i + width, n);
            int right = min(i + 2 * width, n);

            merge(arr, temp, left, mid, right);
        }

        // Параллельное копирование обратно в основной массив
        #pragma omp parallel for schedule(static)
        for (int i = 0; i < n; i++)
            arr[i] = temp[i];
    }

    delete[] temp;
}

// ======================================================
// Проверка корректности сортировки
// ======================================================
bool isSorted(const vector<int>& v) {
    for (size_t i = 1; i < v.size(); i++)
        if (v[i - 1] > v[i])
            return false;
    return true;
}

// ======================================================
// Main
// ======================================================
int main() {

    // Размеры массивов для тестирования
    vector<int> sizes = {100, 1000, 10000, 100000, 1000000};

    // Генератор случайных чисел
    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<int> dist(0, 1000000);

    // Тестируем для каждого размера
    for (int n : sizes) {

        // Инициализация массива на CPU
        vector<int> h(n);
        for (int& x : h)
            x = dist(gen);

        cout << "========== Размер массива: " << n << " ==========\n";

        // Вывод первых и последних элементов до сортировки
        cout << "До сортировки (первые 10): ";
        for (int i = 0; i < min(10, n); i++) cout << h[i] << " ";
        cout << "\nДо сортировки (последние 10): ";
        for (int i = max(0, n - 10); i < n; i++) cout << h[i] << " ";
        cout << "\n";

        // Выделение памяти на GPU
        int* d;
        cudaMalloc(&d, n * sizeof(int));

        // Копирование данных CPU → GPU
        cudaMemcpy(d, h.data(), n * sizeof(int), cudaMemcpyHostToDevice);

        // Конфигурация CUDA
        int blockSize = 1024; // максимальный размер блока
        int blocks = (n + blockSize - 1) / blockSize;

        // Замер времени
        auto start = chrono::high_resolution_clock::now();

        // --------------------------------------------------
        // Этап 1: CUDA
        // Сортировка локальных подмассивов на GPU
        // --------------------------------------------------
        sortBlocksSharedKernel<<<blocks, blockSize,
            blockSize * sizeof(int)>>>(d, blockSize, n);

        cudaDeviceSynchronize();

        // Копирование результата обратно на CPU
        cudaMemcpy(h.data(), d, n * sizeof(int), cudaMemcpyDeviceToHost);

        // --------------------------------------------------
        // Этап 2: CPU + OpenMP
        // Параллельное слияние отсортированных блоков
        // --------------------------------------------------
        parallelMergeSortOMP(h.data(), n);

        auto end = chrono::high_resolution_clock::now();
        chrono::duration<double, milli> elapsed = end - start;

        // Вывод результата
        cout << "После сортировки (первые 10): ";
        for (int i = 0; i < min(10, n); i++) cout << h[i] << " ";
        cout << "\nПосле сортировки (последние 10): ";
        for (int i = max(0, n - 10); i < n; i++) cout << h[i] << " ";
        cout << "\n";

        cout << "Время сортировки: " << elapsed.count() << " мс\n";
        cout << (isSorted(h)
                 ? "Массив отсортирован корректно\n\n"
                 : "ОШИБКА СОРТИРОВКИ\n\n");

        // Освобождение памяти GPU
        cudaFree(d);
    }

    return 0;
}

Writing task_1_merge_sort.cu


In [3]:
!nvcc -Xcompiler -fopenmp task_1_merge_sort.cu -o task_1_merge_sort
!./task_1_merge_sort

========== Размер массива: 100 ==========
До сортировки (первые 10): 42358 665036 200454 727219 633003 616741 509194 852104 683128 24513 
До сортировки (последние 10): 808992 229879 809487 831747 83562 488466 554762 787414 212941 428916 
После сортировки (первые 10): 3640 4918 24513 30362 42358 68524 70691 79352 80202 83562 
После сортировки (последние 10): 880295 887805 907207 914644 922986 924416 934441 954422 987880 989459 
Время сортировки: 43.6393 мс
Массив отсортирован корректно

========== Размер массива: 1000 ==========
До сортировки (первые 10): 187584 611331 583140 957383 395858 783325 744532 457179 304277 837726 
До сортировки (последние 10): 369970 110926 986340 333375 775698 269784 415074 277988 61768 709374 
После сортировки (первые 10): 305 371 4102 4312 4391 5270 5449 7379 7675 9023 
После сортировки (последние 10): 991685 992180 993632 995689 995916 996091 997256 998142 998494 999171 
Время сортировки: 0.131502 мс
Массив отсортирован корректно

========== Размер массив

# Task 2. Реализовать параллельную быструю сортировку на CUDA

In [9]:
%%writefile task_2_quick_sort_v1.cu
#include <iostream>
#include <vector>
#include <algorithm>
#include <omp.h>
#include <cuda_runtime.h>
#include <iomanip>
#include <random>
#include <chrono>

using namespace std;

// =======================================================
// CUDA kernel: iterative quick sort per thread
// =======================================================
__global__ void kernel_quicksort(float* data, int n, int items_per_thread) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int left = tid * items_per_thread;
    int right = left + items_per_thread - 1;

    if (left < n) {
        if (right >= n) right = n - 1;

        int stack[64];
        int top = -1;
        stack[++top] = left;
        stack[++top] = right;

        while (top >= 0) {
            int r = stack[top--];
            int l = stack[top--];
            if (l >= r) continue;

            float pivot = data[(l + r) / 2];
            int i = l, j = r;

            while (i <= j) {
                while (data[i] < pivot) i++;
                while (data[j] > pivot) j--;
                if (i <= j) {
                    float tmp = data[i];
                    data[i] = data[j];
                    data[j] = tmp;
                    i++; j--;
                }
            }
            if (l < j) { stack[++top] = l; stack[++top] = j; }
            if (i < r) { stack[++top] = i; stack[++top] = r; }
        }
    }
}

// =======================================================
// Utility: print first & last 10 elements
// =======================================================
void print_edges(const vector<float>& arr) {
    int n = arr.size();
    int k = min(10, n);

    cout << "Первые 10: ";
    for (int i = 0; i < k; ++i)
        cout << (int)arr[i] << " ";
    cout << endl;

    cout << "Последние 10: ";
    for (int i = n - k; i < n; ++i)
        cout << (int)arr[i] << " ";
    cout << endl;
}

bool is_sorted_array(const vector<float>& arr) {
    for (size_t i = 1; i < arr.size(); ++i)
        if (arr[i] < arr[i - 1])
            return false;
    return true;
}

// =======================================================
// Test runner
// =======================================================
void run_test(int n) {
    vector<float> h_data(n);

    // === Correct random generation ===
    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<> dist(-100000, 100000);

    #pragma omp parallel for
    for (int i = 0; i < n; ++i)
        h_data[i] = dist(gen);

    cout << "\n====================================\n";
    cout << "Размер массива: " << n << endl;

    cout << "До сортировки:\n";
    print_edges(h_data);

    float* d_data;
    cudaMalloc(&d_data, n * sizeof(float));
    cudaMemcpy(d_data, h_data.data(), n * sizeof(float), cudaMemcpyHostToDevice);

    int threads = 1024;
    int items_per_thread = (n + threads - 1) / threads;

    // === Time measurement with chrono ===
    auto start = chrono::high_resolution_clock::now();

    kernel_quicksort<<<1, threads>>>(d_data, n, items_per_thread);
    cudaDeviceSynchronize();

    cudaMemcpy(h_data.data(), d_data, n * sizeof(float), cudaMemcpyDeviceToHost);

    // Final global sort
    sort(h_data.begin(), h_data.end());

    auto end = chrono::high_resolution_clock::now();
    chrono::duration<double, milli> elapsed = end - start;

    cudaFree(d_data);

    cout << "После сортировки:\n";
    print_edges(h_data);

    cout << "Время выполнения: " << fixed << setprecision(3)
         << elapsed.count() << " мс" << endl;

    cout << "Массив отсортирован: "
         << (is_sorted_array(h_data) ? "ДА" : "НЕТ") << endl;
}

// =======================================================
// Main
// =======================================================
int main() {
    vector<int> sizes = {100, 1000, 10000, 100000, 1000000};
    for (int n : sizes)
        run_test(n);
    return 0;
}

Writing task_2_quick_sort_v1.cu


In [10]:
!nvcc task_2_quick_sort_v1.cu -o task_2_quick_sort_v1
!./task_2_quick_sort_v1


Размер массива: 100
До сортировки:
Первые 10: -82182 -79009 9183 -48886 -63251 -36889 84106 84322 -72443 -66596 
Последние 10: 24270 -87853 53861 -7314 -45401 72880 15646 11572 -68386 -86080 
После сортировки:
Первые 10: -96798 -96626 -95880 -89186 -87853 -86320 -86080 -84586 -83254 -82182 
Последние 10: 86845 88466 88532 88754 88876 89902 90226 91509 94229 96780 
Время выполнения: 7.840 мс
Массив отсортирован: ДА

Размер массива: 1000
До сортировки:
Первые 10: -38875 64593 -10722 -89397 47122 89228 -21083 -36939 23540 15370 
Последние 10: -75338 87075 -44493 89236 -67736 -77415 -19422 -38291 -26123 -72969 
После сортировки:
Первые 10: -99910 -99780 -98973 -98968 -98834 -98097 -98016 -97524 -97472 -97463 
Последние 10: 98132 98625 98771 98901 99160 99415 99473 99594 99625 99664 
Время выполнения: 0.270 мс
Массив отсортирован: ДА

Размер массива: 10000
До сортировки:
Первые 10: -80479 85029 17464 34106 -70452 -9257 9481 -65853 69657 77057 
Последние 10: 96577 43895 67641 20420 96001 38

In [11]:
%%writefile task_2_quick_sort_v2.cu
// ======================================================
// Гибридная быстрая сортировка (Quick Sort):
// подсчёт элементов на GPU + разбиение и рекурсия на CPU
// ======================================================

// Подключение стандартной библиотеки ввода-вывода
#include <iostream>

// Подключение контейнера vector
#include <vector>

// Подключение стандартных алгоритмов (swap, min, max)
#include <algorithm>

// Подключение генератора случайных чисел
#include <random>

// Подключение библиотеки для измерения времени
#include <chrono>

// Подключение CUDA Runtime API
#include <cuda_runtime.h>

// Используем стандартное пространство имён
using namespace std;

// ======================================================
// Макрос для проверки ошибок CUDA
// ======================================================
// Проверяет результат CUDA-вызова и завершает программу
// при возникновении ошибки
#define CUDA_SAFE(call)                                      \
    do {                                                     \
        cudaError_t err = call;                              \
        if (err != cudaSuccess) {                            \
            cerr << "CUDA error: "                           \
                 << cudaGetErrorString(err)                  \
                 << " at line " << __LINE__ << endl;         \
            exit(EXIT_FAILURE);                              \
        }                                                    \
    } while (0)

// ======================================================
// CUDA-ядро: подсчёт элементов меньше опорного
// ======================================================
// d_array   — массив данных на устройстве (GPU)
// pivot     — опорный элемент
// d_counter — счётчик элементов меньше pivot
// size      — размер обрабатываемого массива
__global__ void gpuCountLower(const int* d_array,
                              int pivot,
                              int* d_counter,
                              int size) {

    // Глобальный индекс потока
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверяем, что индекс не выходит за пределы массива
    // и элемент меньше опорного
    if (tid < size && d_array[tid] < pivot) {
        // Атомарно увеличиваем счётчик
        atomicAdd(d_counter, 1);
    }
}

// ======================================================
// Гибридная быстрая сортировка (Quick Sort)
// ======================================================
// data  — сортируемый вектор
// left  — левая граница
// right — правая граница
void hybridQuickSort(vector<int>& data, int left, int right) {

    // Базовый случай рекурсии
    if (left >= right) return;

    // Выбор опорного элемента (последний элемент)
    int pivotValue = data[right];

    // Размер текущего сегмента
    int segmentSize = right - left + 1;

    // Указатели на память GPU
    int* d_segment = nullptr;
    int* d_lessCount = nullptr;

    // Выделение памяти на GPU для сегмента массива
    CUDA_SAFE(cudaMalloc(&d_segment, segmentSize * sizeof(int)));

    // Выделение памяти на GPU для счётчика
    CUDA_SAFE(cudaMalloc(&d_lessCount, sizeof(int)));

    // Копирование сегмента массива с CPU на GPU
    CUDA_SAFE(cudaMemcpy(d_segment,
                         &data[left],
                         segmentSize * sizeof(int),
                         cudaMemcpyHostToDevice));

    // Обнуление счётчика на GPU
    CUDA_SAFE(cudaMemset(d_lessCount, 0, sizeof(int)));

    // Настройка конфигурации CUDA
    dim3 threads(256);  // количество потоков в блоке
    dim3 blocks((segmentSize + threads.x - 1) / threads.x); // количество блоков

    // Запуск CUDA-ядра для подсчёта элементов
    gpuCountLower<<<blocks, threads>>>(d_segment,
                                       pivotValue,
                                       d_lessCount,
                                       segmentSize);

    // Ожидание завершения выполнения ядра
    CUDA_SAFE(cudaDeviceSynchronize());

    // ---- Разбиение массива на CPU ----
    int storeIndex = left - 1;

    // Проход по массиву и перестановка элементов
    for (int j = left; j < right; ++j) {
        if (data[j] < pivotValue) {
            ++storeIndex;
            swap(data[storeIndex], data[j]);
        }
    }

    // Перемещение опорного элемента на своё место
    swap(data[storeIndex + 1], data[right]);

    // Индекс опорного элемента после разбиения
    int pivotIndex = storeIndex + 1;

    // Освобождение памяти GPU
    CUDA_SAFE(cudaFree(d_segment));
    CUDA_SAFE(cudaFree(d_lessCount));

    // Рекурсивная сортировка левой части
    hybridQuickSort(data, left, pivotIndex - 1);

    // Рекурсивная сортировка правой части
    hybridQuickSort(data, pivotIndex + 1, right);
}

// ======================================================
// Проверка, отсортирован ли массив
// ======================================================
bool checkSorted(const vector<int>& data) {

    // Проверяем, что каждый элемент не больше следующего
    for (size_t i = 1; i < data.size(); ++i)
        if (data[i - 1] > data[i])
            return false;

    return true;
}

// ======================================================
// Вывод первых и последних 10 элементов массива
// ======================================================
void printEdges(const vector<int>& data) {

    // Размер массива
    int n = data.size();

    // Вывод первых 10 элементов
    cout << "Первые 10: ";
    for (int i = 0; i < min(10, n); ++i)
        cout << data[i] << " ";

    // Вывод последних 10 элементов
    cout << "\nПоследние 10: ";
    for (int i = max(0, n - 10); i < n; ++i)
        cout << data[i] << " ";

    cout << "\n";
}

// ======================================================
// Главная функция
// ======================================================
int main() {

    // Набор размеров массивов для тестирования
    vector<int> testSizes = {100, 1000, 10000, 100000, 1000000};

    // Инициализация генератора случайных чисел
    mt19937 rng(random_device{}());

    // Диапазон случайных значений
    uniform_int_distribution<int> dist(-100000, 100000);

    // Цикл по всем размерам массивов
    for (int size : testSizes) {

        // Создание массива заданного размера
        vector<int> values(size);

        // Заполнение массива случайными числами
        for (int& v : values)
            v = dist(rng);

        // Вывод разделителя
        cout << "====================================\n";

        // Вывод размера массива
        cout << "Размер массива: " << size << "\n";

        // Вывод массива до сортировки
        cout << "До сортировки:\n";
        printEdges(values);

        // Засечение времени начала
        auto startTime = chrono::high_resolution_clock::now();

        // Запуск гибридной сортировки
        hybridQuickSort(values, 0, size - 1);

        // Засечение времени окончания
        auto endTime = chrono::high_resolution_clock::now();

        // Вычисление времени выполнения
        chrono::duration<double, milli> elapsed = endTime - startTime;

        // Вывод массива после сортировки
        cout << "После сортировки:\n";
        printEdges(values);

        // Вывод времени выполнения
        cout << "Время выполнения: "
             << elapsed.count() << " мс\n";

        // Проверка корректности сортировки
        cout << "Массив отсортирован: "
             << (checkSorted(values) ? "ДА" : "НЕТ") << "\n";
    }

    // Завершение программы
    return 0;
}

Writing task_2_quick_sort_v2.cu


In [12]:
!nvcc task_2_quick_sort_v2.cu -o task_2_quick_sort_v2
!./task_2_quick_sort_v2

Размер массива: 100
До сортировки:
Первые 10: 50909 -42550 -70993 7769 11175 4835 97624 76710 59496 11280 
Последние 10: -32164 57668 6010 41737 76000 -20540 -82636 -14395 -67880 -54568 
После сортировки:
Первые 10: -99901 -97520 -96488 -87064 -82998 -82636 -82216 -76203 -75511 -70993 
Последние 10: 76710 80171 83794 85441 92856 95253 96877 97624 98373 99159 
Время выполнения: 114.482 мс
Массив отсортирован: ДА
Размер массива: 1000
До сортировки:
Первые 10: 5682 -99788 -24786 -96654 8769 99155 57642 59028 19175 80246 
Последние 10: 32400 53024 -4043 28735 -1058 -24356 -45775 82348 -46918 -90057 
После сортировки:
Первые 10: -99926 -99788 -99489 -98740 -98735 -98471 -98445 -98376 -98283 -98063 
Последние 10: 98632 98776 99078 99155 99170 99350 99423 99628 99798 99817 
Время выполнения: 81.3407 мс
Массив отсортирован: ДА
Размер массива: 10000
До сортировки:
Первые 10: -63838 -71577 59251 -46624 -76141 -38560 62199 -2169 -87421 -43421 
Последние 10: -61878 1658 13117 52076 72041 288 33880

# Task 3. Реализовать параллельную пирамидальную сортировку на CUDA

In [25]:
%%writefile task_3_heap_sort.cu
// ======================================================
// Parallel GPU Heap Sort (hybrid block heaps)
// ======================================================

// Подключаем стандартные библиотеки
#include <iostream>      // Для ввода/вывода (cout, cerr)
#include <vector>        // Для динамических массивов std::vector
#include <random>        // Для генерации случайных чисел
#include <chrono>        // Для измерения времени выполнения
#include <cuda_runtime.h> // Основной заголовок CUDA для работы с GPU
#include <algorithm>     // Для std::swap и std::min

using namespace std;    // Чтобы не писать std:: перед каждым объектом/функцией

// ======================================================
// Макрос для проверки ошибок CUDA
// ======================================================
#define CUDA_CHECK(call) do { \
    cudaError_t err = call; /* Выполняем вызов CUDA */ \
    if (err != cudaSuccess) { /* Если есть ошибка */ \
        cerr << "CUDA error: " << cudaGetErrorString(err) /* Выводим сообщение об ошибке */ \
             << " at line " << __LINE__ << endl; /* Указываем строку */ \
        exit(EXIT_FAILURE); /* Завершаем программу */ \
    } \
} while(0)

// ======================================================
// GPU kernel: max-heapify для сегмента блока
// ======================================================
__global__ void heapify_block(int *arr, int n, int blockSize) {
    int blockStart = blockIdx.x * blockSize;               // Начальный индекс блока
    int blockEnd = min(blockStart + blockSize, n);        // Конечный индекс блока (не выходим за предел массива)

    // Цикл для heapify каждого узла внутри блока
    // В данном случае один поток проходит весь блок
    for (int i = blockEnd/2 - 1; i >= blockStart; --i) {
        int largest = i;                                  // Изначально считаем текущий узел максимальным
        int l = 2*(i - blockStart) + 1 + blockStart;     // Индекс левого потомка в блоке
        int r = 2*(i - blockStart) + 2 + blockStart;     // Индекс правого потомка в блоке

        if (l < blockEnd && arr[l] > arr[largest]) largest = l; // Если левый потомок больше — обновляем
        if (r < blockEnd && arr[r] > arr[largest]) largest = r; // Если правый потомок больше — обновляем

        if (largest != i) {                               // Если корень не является наибольшим
            int tmp = arr[i];                             // Меняем элементы местами
            arr[i] = arr[largest];
            arr[largest] = tmp;
        }
    }
}

// ======================================================
// CPU Heapify для полной сортировки
// ======================================================
void heapify_cpu(vector<int>& arr, int n, int i) {
    int largest = i;          // Изначально считаем текущий элемент максимальным
    int l = 2*i + 1;          // Индекс левого потомка
    int r = 2*i + 2;          // Индекс правого потомка

    if (l < n && arr[l] > arr[largest]) largest = l; // Проверка левого потомка
    if (r < n && arr[r] > arr[largest]) largest = r; // Проверка правого потомка

    if (largest != i) {       // Если наибольший не корень
        swap(arr[i], arr[largest]);       // Меняем местами
        heapify_cpu(arr, n, largest);     // Рекурсивно продолжаем heapify
    }
}

// ======================================================
// CPU HeapSort для слияния локальных куч
// ======================================================
void heapSort_cpu(vector<int>& arr) {
    int n = arr.size();                  // Размер массива

    // Построение максимальной кучи
    for (int i = n/2 -1; i>=0; --i)
        heapify_cpu(arr, n, i);          // Heapify каждого родителя

    // Последовательное извлечение максимума
    for (int i=n-1; i>0; --i) {
        swap(arr[0], arr[i]);            // Меняем корень с последним элементом
        heapify_cpu(arr, i, 0);          // Heapify для оставшейся части массива
    }
}

// ======================================================
// Гибридная GPU Heap Sort (локальные кучки на GPU + окончательная сортировка на CPU)
// ======================================================
void hybridHeapSort(vector<int>& arr) {
    int n = arr.size();       // Размер массива
    int *d_arr = nullptr;     // Указатель на массив в памяти GPU

    CUDA_CHECK(cudaMalloc(&d_arr, n * sizeof(int)));                  // Выделяем память на GPU
    CUDA_CHECK(cudaMemcpy(d_arr, arr.data(), n * sizeof(int), cudaMemcpyHostToDevice)); // Копируем данные на GPU

    int blockSize = 1024;                    // Размер блока (количество элементов на блок)
    dim3 threads(1);                         // Один поток на блок
    dim3 blocks((n + blockSize -1)/blockSize); // Вычисляем количество блоков

    // Параллельное построение локальных куч в блоках
    heapify_block<<<blocks, threads>>>(d_arr, n, blockSize);
    CUDA_CHECK(cudaDeviceSynchronize());     // Ждем завершения всех потоков

    // Копируем результат обратно на CPU
    CUDA_CHECK(cudaMemcpy(arr.data(), d_arr, n * sizeof(int), cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaFree(d_arr));             // Освобождаем память GPU

    // Полная сортировка на CPU (слияние локальных куч)
    heapSort_cpu(arr);
}

// ======================================================
// Функция вывода первых и последних 10 элементов массива
// ======================================================
void printEdges(const vector<int>& arr) {
    int n = arr.size();                     // Размер массива
    cout << "Первые 10: ";
    for (int i=0;i<min(10,n);++i)          // Вывод первых 10 элементов
        cout<<arr[i]<<" ";
    cout<<"\nПоследние 10: ";
    for (int i=max(0,n-10); i<n;++i)       // Вывод последних 10 элементов
        cout<<arr[i]<<" ";
    cout<<"\n";
}

// ======================================================
// Проверка, отсортирован ли массив
// ======================================================
bool isSorted(const vector<int>& arr) {
    for (size_t i=1;i<arr.size();++i)
        if (arr[i-1]>arr[i]) return false; // Если хотя бы один элемент нарушает порядок — массив не отсортирован
    return true;                           // Иначе — отсортирован
}

// ======================================================
// Главная функция
// ======================================================
int main() {
    vector<int> testSizes = {100, 1000, 10000, 100000, 1000000}; // Размеры тестовых массивов

    mt19937 gen(random_device{}());           // Генератор случайных чисел
    uniform_int_distribution<int> dist(-100000,100000); // Диапазон значений

    for (int n:testSizes) {                   // Проходим по каждому размеру массива
        vector<int> arr(n);                   // Создаем массив
        for (int &x:arr) x=dist(gen);         // Заполняем случайными числами

        cout<<"====================================\n";
        cout<<"Размер массива: "<<n<<"\n";
        cout<<"До сортировки:\n";
        printEdges(arr);                      // Вывод первых и последних 10 элементов до сортировки

        auto start = chrono::high_resolution_clock::now(); // Начало измерения времени

        // ==== Гибридная GPU Heap Sort ====
        hybridHeapSort(arr);

        auto end = chrono::high_resolution_clock::now();   // Конец измерения времени
        chrono::duration<double,milli> elapsed = end-start; // Вычисляем длительность в миллисекундах

        cout<<"После сортировки:\n";
        printEdges(arr);                      // Вывод первых и последних 10 элементов после сортировки
        cout<<"Время выполнения: "<<elapsed.count()<<" мс\n"; // Вывод времени
        cout<<"Массив отсортирован: "<<(isSorted(arr)?"ДА":"НЕТ")<<"\n"; // Проверка правильности сортировки
    }

    return 0; // Успешное завершение программы
}


Overwriting task_3_heap_sort.cu


In [26]:
!nvcc task_3_heap_sort.cu -o task_3_heap_sort
!./task_3_heap_sort

Размер массива: 100
До сортировки:
Первые 10: 57019 -82377 44453 32391 -45445 23703 61005 -31290 -89859 59430 
Последние 10: -24639 91141 -60633 -28139 89162 33400 33343 -74175 -69605 60679 
После сортировки:
Первые 10: -98258 -91439 -90935 -90068 -89859 -89805 -88936 -86789 -84206 -82377 
Последние 10: 79665 80645 80845 88342 88389 89162 89438 91141 97521 97694 
Время выполнения: 119.579 мс
Массив отсортирован: ДА
Размер массива: 1000
До сортировки:
Первые 10: 41472 8510 28531 -80800 -94719 -22189 -21351 -68800 -51439 72026 
Последние 10: 87609 87223 65711 -33207 45076 74602 -78329 -11741 32233 -6582 
После сортировки:
Первые 10: -99727 -99568 -99400 -99230 -99215 -98591 -98512 -98461 -98231 -98197 
Последние 10: 97490 97827 98231 98279 98344 98490 98611 98673 99178 99747 
Время выполнения: 0.714622 мс
Массив отсортирован: ДА
Размер массива: 10000
До сортировки:
Первые 10: -84052 -27361 -36747 -86445 25129 -5728 -47222 -19728 50380 89009 
Последние 10: -38644 -18713 53475 71666 18176 